In [14]:
import os
import pandas as pd
import nltk
nltk.download('punkt')
import gensim
import numpy as np
from gensim import corpora, models, similarities
from gensim.models import Word2Vec
import logging
import gzip

[nltk_data] Downloading package punkt to /home/rizilab/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
df=pd.read_json('./file.json')

In [16]:
x = df['title'].values.tolist()
y = df['content'].values.tolist()

In [17]:
corpus = x+y

In [18]:
tok_corp = [nltk.word_tokenize(sent) for sent in corpus]

In [20]:
model = Word2Vec(tok_corp,min_count=1,size=32)

In [31]:
w1 = "the"
model.wv.most_similar(positive=w1)

[(',', 0.9998342990875244),
 ('.', 0.9998197555541992),
 ('that', 0.99980628490448),
 ('of', 0.9997879862785339),
 ('to', 0.9997819066047668),
 ('in', 0.9997571706771851),
 ('and', 0.9997542500495911),
 ('a', 0.9997342228889465),
 ('with', 0.9997031688690186),
 ('was', 0.9996986985206604)]

In [22]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

In [23]:
def visualize(model, output_path):
    meta_file = "w2x_metadata.tsv"
    placeholder = np.zeros((len(model.wv.index2word), 32))

    with open(os.path.join(output_path,meta_file), 'wb') as file_metadata:
        for i, word in enumerate(model.wv.index2word):
            placeholder[i] = model[word]
            # temporary solution for https://github.com/tensorflow/tensorflow/issues/9094
            if word == '':
                print("Emply Line, should replecaed by any thing else, or will cause a bug of tensorboard")
                file_metadata.write("{0}".format('<Empty Line>').encode('utf-8') + b'\n')
            else:
                file_metadata.write("{0}".format(word).encode('utf-8') + b'\n')

    # define the model without training
    sess = tf.InteractiveSession()

    embedding = tf.Variable(placeholder, trainable = False, name = 'w2x_metadata')
    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(output_path, sess.graph)

    # adding into projector
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = 'w2x_metadata'
    embed.metadata_path = meta_file

    # Specify the width and height of a single thumbnail.
    projector.visualize_embeddings(writer, config)
    saver.save(sess, os.path.join(output_path,'w2x_metadata.ckpt'))
    print('Run `tensorboard --logdir={0}` to run visualize result on tensorboard'.format(output_path))

In [24]:
# visualize(model,"./")

In [25]:
model.save('model_cnn_edition')

In [26]:
len(model.wv.vocab)

3826